# Lecture 2

## 2.1 Jupyter notebooks

In [ ]:
import sys
print(sys.executable)
!which python3

In [ ]:
!pip --version
%pip --version

In [ ]:
%pip install flask
import flask

In [ ]:
import site
print(site.getsitepackages())
!ls -l {site.getsitepackages()[0]}

In [ ]:
%lsmagic

In [ ]:
import eccodes

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f = open("../e-ai_ml2/course/code/code03/icon_t2m.grib", "rb")

In [ ]:
%pip install cartopy

In [ ]:
import cartopy.crs as ccrs

## GPU access in practice

In [ ]:
import torch

In [ ]:
print(torch.cuda.is_available())

In [ ]:
print(torch.backends.mps.is_available())

In [ ]:
import time

In [ ]:
d = torch.device("mps")

In [ ]:
x = torch.rand((40000,40000),device=d)

In [ ]:
t0 = time.time()
y = torch.matmul(x, x)
torch.mps.synchronize()
print("Time = ", round(time.time()-t0,3))

In [ ]:
n = 40000
x0 = torch.rand((n, n), device="cpu")
x1 = torch.rand((n, n), device="cpu")
t0 = time.time()
y = torch.matmul(x0, x1)
print("Time = ", round(time.time() - t0, 3))

### Mixed precision

In [ ]:
%pip install wget

In [ ]:
import wget

In [ ]:
%pip --version

In [ ]:
!pip --version

## AI and ML

### Torch tensors

In [ ]:
import torch
x = torch.tensor([2., 3.], requires_grad=True)
y = x[0]**2 + x[1]**2
y.backward()
print(x.grad)

In [ ]:
import torch.nn as nn

# nn.Module is the base class for models and layers
# Holds parameters (weights and biases)
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layer 1: 1 -> 16
        self.fc1 = nn.Linear(1,16)
        
        # Non-linear activation function (ReLU in this case)
        self.relu = nn.reLU()
        
        # Layer 2: 16 -> 1
        self.fc2 = nn.Linear(16,1)

    # Calling `model(x)` runs the model's `forward()` method
    # Forward pass computes predictions from inputs (x)
    # Builds the autograd graph (if grads enables on x)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)

Learning a sine function

In [ ]:
# Sample input values x
x = np.linspace(0, 2*np.pi, 1000)

# Compute labels y = sin(x)
y = np.sin(x)

plt.plot(x, y)
plt.show()

In [ ]:
# Dataset construction

from torch.utils.data import TensorDataset, DataLoader

x_t = torch.tensor(x).float().unsqueeze(1)
y_t = torch.tensor(y).float().unsqueeze(1)

data = TensorDataset(x_t, y_t)
loader = DataLoader(data,
                    batch_size=32,
                    shuffle=True)

In [ ]:
# Model and training loop

# Learn non-linear mapping x -> \hat{y}
# Input: scalar x
# Output: scalar \hat{y}

# Model
model = nn.Sequential(
    nn.Linear(1,16), nn.ReLU(),
    nn.Linear(16,16), nn.ReLU(),
    nn.Linear(16,1)
)

# Loss function
loss_fn = nn.MSELoss()

# Optimiser
opt = torch.optim.Adam(
    model.parameters(),
    lr = 0.01
)

# Training loop
#     - Compare \hat{y} and y
#     - Minimise prediction error
#     - Update model parameters
for x_b, y_b in loader:
    
    # Zero the gradients from the previous iteration
    opt.zero_grad()

    # Forward pass of the model to get predictions
    y_p = model(x_b)

    # Update loss given predictions y_p
    loss = loss_fn(y_p, y_b)

    # Backpropagation - compute gradients of loss wrt parameters
    loss.backward()

    # Optimiser - update parameters (weights and biases) in-place
    # given the gradients
    opt.step()